In [ ]:
import os, sys
from collections import defaultdict
from urllib.request import urlopen
import json

import numpy as np
import pandas as pd

import plotly.graph_objects as go
from plotly.subplots import make_subplots
from ipywidgets import widgets

In [ ]:
#Adapted from: https://www.kaggle.com/para24/eda-with-plotly
#Retrieved on: Nov 19,2020
df = pd.read_excel('..\\Data\\NewData.xlsx', sheet_name ='Sheet1' )
#df = pd.read_excel('NewData.xlsx', sheet_name ='Sheet1' ) #reading data
#df = pd.read_csv('GOI.csv')
df.columns = ['Category', 'Name', 'Y2006', 'Y2010',
              'Y2014', 'Y2018']
#df.head(20)

In [ ]:
df_cols=["Y2006", "Y2010", "Y2014", "Y2018"]

#converting values to int and filling NA as 0 for calc
for value in df_cols:
    temp = df[value]
    temp = temp.fillna(0)
    temp = temp.astype(int)
    df[value] = temp

#taking difference to get change in population    
df['D2010'] = df['Y2010'].sub(df['Y2006'], axis = 0)
df['D2014'] = df['Y2014'].sub(df['Y2010'], axis = 0) 
df['D2018'] = df['Y2018'].sub(df['Y2014'], axis = 0)

#removing original populations as not needed now
del df["Y2006"]
del df["Y2010"]
del df["Y2014"]
del df["Y2018"]

#sorting by state
df = pd.wide_to_long(df, ['D'], i="Name", j="Year")
df.sort_values(by=['Category', 'Name'], inplace=True)
df.reset_index(inplace=True)
df.head(20)

In [ ]:
#Read the polygon information of various Indian states from the GeoJSON file
with open('..\\Data\\india_states.geojson', 'r') as fp:
#with open('india_states.geojson', 'r') as fp:
    india = json.load(fp)

In [ ]:
#matching state names in geoJson with state names in data
geo_df = pd.DataFrame(data=[st['properties']['NAME_1'] for st in india['features']], columns=['State Names in GeoJSON'])
data_df = pd.DataFrame(df.iloc[2:, 0].unique(), columns=['State Names in DATA'])
geo_df.merge(data_df,
             how='outer',
             left_on='State Names in GeoJSON',
             right_on='State Names in DATA')

In [ ]:
#fixing remaining names to get matches in GeoJson
mapper = {'Jammu & Kashmir': 'Jammu and Kashmir',
          'Odisha': 'Orissa',
          'Uttarakhand': 'Uttaranchal',
          'A & N Islands': 'Andaman and Nicobar',
          'D & N Haveli': 'Dadra and Nagar Haveli',
          'Daman & Diu': 'Daman and Diu',
          'NCT of Delhi':'Delhi'}
df.iloc[:, 0] = df.iloc[:, 0].apply(lambda s: mapper[s] if s in mapper.keys() else s)

In [ ]:
# Check to confirm if the names are mapped properly
#print(sorted([st['properties']['NAME_1'] for st in india['features']]) == sorted(list(df.iloc[2:, 0].unique())))

data_df = pd.DataFrame(df.iloc[4:, 0].unique(), columns=['State Names in DATA'])
geo_df.merge(data_df, how='outer',
             left_on='State Names in GeoJSON',
             right_on='State Names in DATA')

In [ ]:
#grouping data by year for plotting
df_grouped = df.groupby(by='Year')
#for key, item in df_grouped:
    #print(df_grouped.get_group(key), "\n\n")

In [ ]:
#Adapted from: https://towardsdatascience.com/discrete-colour-scale-in-plotly-python-26f2d6e21c77
#Retrieved on: Dec 09,2020
#generating colour scale for discrete numbers
def generateDiscreteColourScale(colour_set):
    #colour set is a list of lists
    colour_output = []
    num_colours = len(colour_set)
    divisions = 1./num_colours
    c_index = 0.
    # Loop over the colour set
    for cset in colour_set:
        num_subs = len(cset)
        sub_divisions = divisions/num_subs
        # Loop over the sub colours in this set
        for subcset in cset:
            colour_output.append((c_index,subcset))
            colour_output.append((c_index + sub_divisions-
                .001,subcset))
            c_index = c_index + sub_divisions
    colour_output[-1]=(1,colour_output[-1][1])
    return colour_output

color_schemes = [
    ['#b2182b','#ef8a62','#fddbc7'],
    ['#d1e5f0','#67a9cf','#2166ac']
]

colorscale = generateDiscreteColourScale(color_schemes)

#adding white to colorscale for zero population states
colorscale.insert(6, (0.498, '#f7f7f7'))
colorscale.insert(7, (0.502, '#f7f7f7'))
colorscale[5]=(0.497, '#fddbc7')
colorscale[8]=(0.503, '#d1e5f0')
#print(colorscale)

In [ ]:
#creating maps for each year
df_curr = df_grouped.get_group(2010).reset_index(drop=True)
#print(df_curr)
trace1 = go.Choroplethmapbox(geojson=india,
                            featureidkey='properties.NAME_1',
                            locations=df_curr.loc[1:, 'Name'],
                            z=df_curr.loc[1:, 'D'], 
                            zmin=-150,
                            zmax=150,
                            name="2006-2010",
                            colorscale=colorscale,
                            colorbar=dict(title='Change in Tiger Population',
                                          len=0.8,
                                          lenmode='fraction')
                           )


df_curr = df_grouped.get_group(2014).reset_index(drop=True)
trace2 = go.Choroplethmapbox(geojson=india,
                            featureidkey='properties.NAME_1',
                            locations=df_curr.loc[1:, 'Name'],
                            z=df_curr.loc[1:, 'D'], 
                            zmin=-150,
                            zmax=150,
                            name="2010-2014",
                            visible=False,
                            colorscale=colorscale,                           
                            colorbar=dict(title='Change in Tiger Population',
                                          len=0.8,
                                          lenmode='fraction')
                           )


df_curr = df_grouped.get_group(2018).reset_index(drop=True)
trace3 = go.Choroplethmapbox(geojson=india,
                            featureidkey='properties.NAME_1',
                            locations=df_curr.loc[1:, 'Name'],
                            z=df_curr.loc[1:, 'D'], 
                            zmin=-150,
                            zmax=150,
                            name="2014-2018",
                            visible=False,
                            colorscale=colorscale,
                            colorbar=dict(title='Change in Tiger Population',
                                          len=0.8,
                                          lenmode='fraction')
                           )

#creating layout
lyt = dict(title='Change in Tiger Population in India over periods of 4 years',
           height=700,
           mapbox_style='white-bg',
           mapbox_zoom=3.4,
           mapbox_center={'lat': 20.5937, 'lon': 78.9629})

#adding all maps to figure
fig1 = go.FigureWidget()
fig1.add_trace(trace1)
fig1.add_trace(trace2)
fig1.add_trace(trace3)

fig1.layout =  lyt

#adding buttons to select year
fig1.update_layout(
    updatemenus=[go.layout.Updatemenu(
        active=0, # which button is considered active, index starting 0
        type="buttons",
        buttons=list(
            [dict(label = '2006-2010',
                  method = 'update',
                  args = [{'visible': [True, False, False]},
                          {'title': 'Change in Tiger Population in India from 2006-2010',
                           'showlegend':True}]),
             dict(label = '2010-2014',
                  method = 'update',
                  args = [{'visible': [False, True, False]},
                          {'title': 'Change in Tiger Population in India from 2010-2014',
                           'showlegend':True}]),            
             dict(label = '2014-2018',
                  method = 'update',
                  args = [{'visible': [False, False, True]},
                          {'title': 'Change in Tiger Population in India from 2014-2018',
                           'showlegend':True}]),
                         ])
        )
    ])

fig1.show()

In [ ]:
import plotly.io as pio
pio.write_html(fig1, file ='../HTMLs/idiom2.html', auto_open=False)